In [1]:
from pathlib import Path
import json
import socket

import pandas as pd
import geopandas as gpd

from pyagnps import soil_data_market as sdm

from sqlalchemy import URL, create_engine, text as sql_text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

In [2]:
# DATABASE SETUP
credentials = Path("../../inputs/db_credentials.json")
with open(credentials, "r") as f:
    credentials = json.load(f)

user = credentials["user"]
password = credentials["password"]
host = credentials["host"]
port = credentials["port"]
database = credentials["database"]

url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

# create a SQLAlchemy engine object
engine = create_engine(url_object)

nodename = socket.gethostname()

In [3]:
raster_CDL_path = Path('D:/AIMS/Datasets/Management/CDL_Annual/CDL_2022.tif')
path_to_management_class_names = Path('../../inputs/management/CDL_Field_ID_dictionary.csv')


In [4]:
df_cdl = pd.read_csv(path_to_management_class_names)

In [5]:
df_cdl.head()

,CDL_Value,CDL_Category,Modified_CDL_Category,Field_ID,Landuse_Type_ID,Mgmt_Schd_ID
0,1,Corn,Corn,Corn,CROPLAND,Corn
1,2,Cotton,Cotton,Cotton,CROPLAND,Cotton
2,3,Rice,Rice,Rice,CROPLAND,Rice
3,4,Sorghum,Sorghum,Sorghum,CROPLAND,Sorghum
4,5,Soybeans,Soybeans,Soybeans,CROPLAND,Soybeans


In [7]:
dico = df_cdl[['CDL_Value','Modified_CDL_Category']].set_index('CDL_Value').to_dict(orient='dict')['Modified_CDL_Category']
# dico = df_cdl.set_index('CDL_Value').loc[:,['Mgmt_Schd_ID']].to_dict()
# dico = 

In [9]:
thuc_id = '0927'

In [10]:
# Collect thuc cells geometry from database

query = f"SELECT * FROM thuc_{thuc_id}_annagnps_cell_ids"

with engine.connect() as conn:
    cells = gpd.read_postgis(sql=sql_text(query), con=conn, geom_col="geom")

utm = cells.estimate_utm_crs()
cells = cells.to_crs(utm)


2023-08-02-09-46-37: Luc-NCCHE-XPS: 0927: Querying cells from database...


In [19]:
# boundary = cells.copy(deep=True)

# boundary['geom'] = boundary['geom'].buffer(0)
# boundary = boundary.unary_union
# boundary = gpd.GeoDataFrame(geometry=[boundary], crs=utm)

# boundary.explore()

Doing the plurality analysis using the raster

In [12]:
cells = sdm.assign_attr_zonal_stats_raster_layer(cells, raster_CDL_path, agg_method='majority', attr='CDL_Value')
cells['CDL_Value'] = cells['CDL_Value'].astype('Int32')
cells.loc[cells['CDL_Value']==0, 'CDL_Value'] = 81 # Set 0 value to 81 = Cloud_No_Data
cells['Mgmt_Field_ID'] = cells['CDL_Value'].map(dico)
cells = cells.rename(columns={"dn": "cell_id"})

c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\rasterstats\io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


In [13]:
cells

,fid,cell_id,geom,CDL_Value,Mgmt_Field_ID
0,1,5811,"POLYGON ((340398.918 4757227.672, 340398.918 4...",143,Mixed_Forest
1,2,6691,"POLYGON ((354558.918 4757017.672, 354558.918 4...",5,Soybeans
2,3,5812,"POLYGON ((340128.918 4757077.672, 340128.918 4...",1,Corn
3,4,5802,"POLYGON ((340248.918 4756777.672, 340248.918 4...",195,Developed
4,5,5793,"POLYGON ((340188.918 4756807.672, 340188.918 4...",143,Mixed_Forest
...,...,...,...,...,...
2204,2205,4202,"POLYGON ((348828.918 4735747.672, 348828.918 4...",143,Mixed_Forest
2205,2206,4201,"POLYGON ((348768.918 4734757.672, 348768.918 4...",143,Mixed_Forest
2206,2207,4192,"POLYGON ((349908.918 4734277.672, 349908.918 4...",121,Developed_Open_Space
2207,2208,4193,"POLYGON ((349068.918 4735687.672, 349068.918 4...",143,Mixed_Forest


In [14]:
cells.explore(column='Mgmt_Field_ID', categorical=True)

In [15]:
data_to_update = cells[["cell_id", "Mgmt_Field_ID"]].to_dict(orient="records")

Set mgmt_field_id colum as text

In [16]:
with engine.connect() as connection:
    try:
        query = f"ALTER TABLE thuc_{thuc_id}_annagnps_cell_data_section ALTER COLUMN mgmt_field_id TYPE TEXT"
        connection.execute(sql_text(query))
        # Commit the transaction explicitly
        connection.commit()

    except Exception as e:
        print(e)
        # Rollback the transaction in case of an error
        connection.rollback()


Populate Data

In [17]:
try:
    # create a session factory
    Session = sessionmaker(bind=engine)
    # create a new session
    session = Session()
    # create a transaction
    transaction = session.begin()

    # execute your update query here
    query = f"UPDATE thuc_{thuc_id}_annagnps_cell_data_section SET Mgmt_Field_ID = :Mgmt_Field_ID WHERE cell_id = :cell_id"
    session.execute(sql_text(query), data_to_update)
    # commit the transaction
    transaction.commit()

except Exception as e:
    # rollback the transaction on error
    transaction.rollback()

finally:
    # close the session
    session.close()

Check that it worked

In [33]:
# query = 'SELECT * FROM usa_valid_soil_layers_data WHERE "Soil_ID" = 568326'
query = f"SELECT * FROM thuc_{thuc_id}_annagnps_cell_data_section ORDER BY cell_id"

df = pd.read_sql_query(sql=sql_text(query), con=engine.connect())
df.head()

,cell_id,soil_id,mgmt_field_id,reach_id,reach_location_code,cell_area,time_of_conc,avg_elevation,rcn_calib_id,avg_land_slope,...,shallow_conc_flow_length,delivery_ratio,constant_usle_c_fctr,constant_usle_p_fctr,all_oc_calib_fctr,all_n_calib_fctr,all_p_calib_fctr,sheet_and_rill_erosion_calib_fctr,gullies_erosion_calib_fctr,input_units_code
0,22,295611,None,2,1,54.18,None,203.59,None,0.27312,...,50.0,None,None,None,None,None,None,None,None,1
1,23,295681,None,2,1,30.42,None,199.45,None,0.41944,...,50.0,None,None,None,None,None,None,None,None,1
2,32,295678,None,3,1,5.40,None,256.17,None,0.05635,...,50.0,None,None,None,None,None,None,None,None,1
3,33,295575,None,3,1,4.14,None,255.01,None,0.05885,...,50.0,None,None,None,None,None,None,None,None,1
4,41,295632,None,4,0,11.07,None,275.41,None,0.03250,...,50.0,None,None,None,None,None,None,None,None,1


In [24]:
df[df['mgmt_field_id'].isnull()].empty

True